# Utayomi
短歌の一覧を指定したLLMに入力し、評を生成するシステムです。  
設計・作成: ef_utakata(https://x.com/ef_utakata)

## ToDo
* 連作入力対応
* 複数LLMの出力をファシリテーションするモード
* 再生成モード（指定番号を再生成）
* コンテナ化

## 対応モデル
以下の形式のモデルに対応しています。
1. huggingface形式のモデル(transformerを使用)
2. gguf形式の量子化モデル(llama-cpp-pythonを使用)
3. openAI APIで利用できるモデル(GPT-4oなど)
4. cohere APIで利用できるモデル(Command r+)
5. google.generativeai APIで利用できるモデル(Gemini-1.5-proなど)

2024年6月29日時点で、以下のモデルを用いた入力短歌へのコメントの出力が可能です。

* Umievo-itr012-Gleipnir-7B
    * https://huggingface.co/umiyuki/Umievo-itr012-Gleipnir-7B/tree/main

* Oumuamua-7b-instruct-v2
    * https://huggingface.co/nitky/Oumuamua-7b-instruct-v2

* Phi-3-mini, Phi-3-medium
    * https://huggingface.co/microsoft

* Ninja-v1-RP  
    * https://huggingface.co/Aratako/Ninja-v1-RP

* Ninja-V2-7B  
    * https://huggingface.co/Local-Novel-LLM-project/Ninja-V2-7B

* Llama-3-elyza-jp-8b
    * https://huggingface.co/elyza/Llama-3-ELYZA-JP-8B-GGUF

* Command-r-plus（API key必要）  
    * https://huggingface.co/CohereForAI/c4ai-command-r-plus

* Gemini(API key必要、1.5-pro, 1.5-flash, 1.0-pro, gemini-pro の4つを自動で切り替え)  
    * https://gemini.google.com/?hl=ja

* GPT-4o（API key必要）  
    * https://platform.openai.com/docs/overview

APIでアクセスするモデルを利用する場合は、それぞれのモデルの配布元からAPI keyを取得し、以下の環境変数に入力する必要があります。
* openAI: OPENAI_API_KEY
* cohere: COHERE_API_KEY
* Google: GOOGLE_API_KEY

pypeline.pyの実行前に、.bashrcに各値を入力するか、以下のコマンドでAPI keyを入力してください。  
notebook上で実行する場合は"API keyの入力"と記載のあるセルにkeyを入力してセルを実行すると一括入力されます。

```bash
export OPENAI_API_KEY="取得したAPI key"
export COHERE_API_KEY="取得したAPI key"
export GOOGLE_API_KEY="取得したAPI key"
```

短歌生成におけるモデルの指定や生成時の詳細な設定は、yaml形式のファイル(model_conf.yaml)で記述します。
引数-i でファイル内のどの設定を読み込むかを指定します。

以下のコマンドで、-i に入力可能な識別子一覧を表示できます。

```bash
python pipeline.py --list 
```

## 設定ファイルmodel_conf.yamlの記述方法
設定ファイル(model_conf.yaml)は利用可能なモデルを追加したり細かい設定を変更する場合などに開発者が編集しやすいようにするためのもので、
システムの利用のみの場合は特に編集する必要はありません。

## 入力ファイルの記述方法
入力フォーマット:csv(UTF-8)ファイル(以下のフォーマットに従って記述されているもの)
* 先頭行(header): No,Content,Author,Author_comment,Human_comment
    * No: 通し番号(1から順番)
    * Content: 短歌
    * Author: 作者名
    * Author_comment: 作者コメント
    * Human_comment: AI評を確認してコメントを付与する場合に使う列(オプション)

自作短歌を用いた入力例はinput/demoディレクトリ内にあります。

* input/ef_test_free.csv: 自由詠
* input/ef_test_theme.csv: 題詠（お題：「海」）
* input/ef_test_theme_sea_human_comment.csv: 自由詠、AI評に対するコメントを入力した例

これらをシステム上の対応モデルに入力して生成したコメントは、output/demoディレクトリ内にあります。

* input/ef_test_free_Ninja-v2-7b.csv: 自由詠
* input/ef_test_theme_Ninja-v2-7b.csv: 題詠（お題：「海」）の例
* input/ef_test_theme_sea_human_comment_Ninja-v2-7b.csv: 題詠の出力されたAI評に対するコメントを入力した例

## 出力ファイルの記述形式
出力フォーマット:csv(UTF-8)
* 先頭行(header):No,Tanka,Author,Author_comment,LLM identifier
* No: 通し番号(1から順番)
* Tanka: 短歌
* Author: 作者名
* Author_comment: 作者コメント
* LLM identifier: LLMによるコメント

## 歌会モード
複数のLLMからのコメントをGeminiに入力し、共通点や相違点についての要約を出力するモードです。  
引数mに"utakai"を指定、iをGeminiに設定し、入力ファイルを各LLMからのコメントが記述されたCSVを指定すると実行されます。
notebook中に実行例を記載しています。  


# tankaAIの環境構築
* 参考:福山大工学部情報工学科 金子邦彦研究室 (2024年6月9日閲覧)  
    * WSL2 上の Ubuntu での NVIDIA CUDA ツールキット, NVIDIA cuDNN, PyTorch, TensorFlow 2.11 のインストールと動作確認（Windows 上）   
        https://www.kkaneko.jp/tools/wsl/wsl_tensorflow2.html  

現在、以下の環境で動作を確認しています。GPUアクセスが可能なWSL(Ubuntu)でも動作すると思いますが、確認はしていません。
* Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
* DRAM 32GB
* NVIDIA RTX A4000 VRAM16GB
* Ubuntu 22.04.3 LTS

環境構築にはUbuntuなどのLinuxのCLI動作に関する知識がある程度必要になります。  
nvidia-driver, cuda, cudnn等のGPUにアクセスするための環境構築は使用機器によって大きく異なるので、ここでは説明しません。  
nvcc -V やnvidia-smiなどのコマンドが動作する環境が本リポジトリを動作させる前提になります。
以下の環境構築は、cuda version 12が動作する前提のコマンドです。
具体的なcuda環境構築の方法については上記リンクのwebページなどを参照してください。  

```bash
# aptで利用する基本パッケージ等のアップデート
sudo apt -y update && sudo apt -y upgrade
sudo apt -y install python3-dev python3-pip python3-setuptools

# minicondaのインストール(インストール済みの場合は不要)
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda.sh
bash ~/miniconda.sh -b -p $HOME/miniconda

# terminalを再起動, condaコマンドが動作することを確認

# tankaAIの仮想環境を構築
conda create -n tankaAI python=3.11 jupyterlab
conda actibate tankaAI

# GPUで動作するpytorchのインストール
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
# pytorchのversion動作確認
python3 -c "import torch; print( torch.__version__ )"
# 2.2.0+cu121
python3 -c "import torch; print(torch.__version__, torch.cuda.is_available())"
# 2.2.0+cu121 True 

# llama-cpp-python(GPU対応)をインストール(cuda-12の場合)
export CUDACXX="/usr/local/cuda-12/bin/nvcc"
export CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=all-major"
export FORCE_CMAKE=1 
pip install llama-cpp-python --no-cache-dir --force-reinstall --upgrade

# 本リポジトリをクローン
git clone ####

# 動作に必要なpythonライブラリのインストール
# $ pip install -r requirements.txt

pip install -r pandas=2.2.0 pyyaml colorama transformers=4.37.2 llama-cpp-python cohere openai google.generativeai
# huggingface-cliのインストール
pip install -U "huggingface_hub[cli]"
```

In [ ]:
# API keyのセット
%env OPENAI_API_KEY=*********************
%env COHERE_API_KEY=*********************
%env GOOGLE_API_KEY=*********************

In [1]:
# 利用方法の概要
!python pipeline.py -h ./input/ef_test.csv ./output/ef_test_output.csv

usage: pipeline.py [-h] [-c CONFIG] [-i IDENTIFIER] [-m MODE] [-t THEME]
                   [--list] [-V]
                   input output

Utayomi: 入力された短歌についてLLMにより評を生成するシステムです。設計: ef_utakata

positional arguments:
  input                 入力短歌一覧のパス(csv形式で入力), 再生成モードの場合は前回の出力一覧
  output                出力先ディレクトリのパス(csv形式で出力)

options:
  -h, --help            show this help message and exit
  -c CONFIG, --config CONFIG
                        利用モデルの入力設定ファイル(yaml形式)
  -i IDENTIFIER, --identifier IDENTIFIER
                        入力設定ファイル内の設定識別子(--listで一覧を確認可能)
  -m MODE, --mode MODE  実行モード{first(初回生成) / utakai(要約)} default: first
  -t THEME, --theme THEME
                        お題(入力がない場合自由詠)
  --list                -cで指定した入力設定ファイルの一覧を表示
  -V, --version         バージョン情報の表示


In [ ]:
# 利用できるモデルとダウンロード済みモデルの一覧表示
!python pipeline.py --list ./input/ef_test.csv ./output/ef_test_output.csv

In [2]:
# 入力例(自由詠+コメント)
import pandas as pd

input_csv = "./input/demo/ef_test_free.csv"
df = pd.read_csv(input_csv)
df

,Content,Author,Author_comment,No
0,いざよひをめづるこころはもちづきのころよりましてなほあかざりし,ef,NaN,1
1,十六夜《いざよひ》を愛づるこころは望月のころより増してなほ飽かざりし,ef,あなたが完璧だとわかったので、私はあなたを愛しました。それから私はあなたが完璧ではないことに...,2
2,友愛も敵意も遠く吹き過ぎて平均すればなだらかな線,ef,それが人生だ、永遠の友人も永遠の敵もいない,3
3,眼下へとひらくみどりは雪原のさやけき白をキャンバスにして,ef,立山の室堂に広がる雪景色とその向こうに見える富山平野の緑を見て。,4
4,きょうという一日を摘むまたあしたこの草原をすすむためにも,ef,NaN,5
5,日輪が地磁気を揺らし水鏡につらなる苗に波紋をのこす,ef,磁気嵐がGPS衛星に障害を起こし、GPSを用いた田植え機を使用している田んぼで苗の列がズレた...,6


In [3]:
# 入力例(題詠「海」、コメントなし)
import pandas as pd

input_csv = "./input/demo/ef_test_theme_sea.csv"
df = pd.read_csv(input_csv)
df

,Content,Author,Author_comment,No
0,海底の傷に海鼠（なまこ）は息づいてふうわりと降る雪を蒐める,ef,NaN,1
1,もう一度魚雷と添い寝できたなら珊瑚の海をあいつと泳ぐ,ef,NaN,2
2,河口から海底に吹く乱流を颪（おろし）と呼ぶのだろうか鯱（しゃち）は,ef,NaN,3
3,東（ひんがし）へ干潟をすすむ引き潮と光の海をひらきつづけて,ef,NaN,4
4,掌にタコが生まれて海がくる 祖母は魚の貌をしている,ef,NaN,5
5,僕らみな墓標となりて茫洋と菩薩のような盆暮れの海,ef,NaN,6


In [4]:
# 入力例(題詠「海」、コメントなし)
import pandas as pd

input_csv = "./input/demo/ef_test_theme_sea_human_comment.csv"

df = pd.read_csv(input_csv, index_col=0)
df

,Content,Author,Author_comment,Comment,Human_comment
No,,,,,
1,海底の傷に海鼠（なまこ）は息づいてふうわりと降る雪を蒐める,ef,NaN,\n\nこの短歌は、まず視覚的な美しさを感じさせます。「海底の傷」というフレーズから、海中世...,「海底の傷」は海底に無数に走る海溝や海嶺のことで、「雪」はマリンスノーのこと。
2,もう一度魚雷と添い寝できたなら珊瑚の海をあいつと泳ぐ,ef,NaN,\n\nこの短歌は、非常に象徴的かつロマンチックな作品です。主題は「海」と「愛する人と共に過...,潜水艦に乗る若手の自衛隊員は格納庫で魚雷の横で睡眠をとることがあるらしい。
3,河口から海底に吹く乱流を颪（おろし）と呼ぶのだろうか鯱（しゃち）は,ef,NaN,\n\nこの短歌は、海とその周辺環境に焦点を当てた深みのある作品です。最初のフレーズ「河口か...,海の生物は、河口を海の中から見上げたときにどのようなことを思うのだろうか。
4,東（ひんがし）へ干潟をすすむ引き潮と光の海をひらきつづけて,ef,NaN,"\n\nこの短歌は、自然の美しさと変化を見事に表現しています。特に注目したいのは、""引き潮と...",全ての句のあたまが「ひ」で始まることばになるような頭韻が技法として施されている。
5,掌にタコが生まれて海がくる 祖母は魚の貌をしている,ef,NaN,\n\nこの短歌は、まず視覚的なイメージが強烈であり、読者の想像力を刺激します。「掌にタコが...,「たこ」は海の蛸と手のひらにできる胼胝をかけている。また、ラブクラフトの小説「インスマウスの...
6,僕らみな墓標となりて茫洋と菩薩のような盆暮れの海,ef,NaN,\n\nこの短歌は、一見すると奇抜な表現が目を引きますが、深く読み込むと海の持つ壮大さや人生...,全ての句のあたまが「ぼ」で始まることばになるような頭韻の技法が採用されている。


In [5]:
# 自由詠でLlama-3-elyza-jp-8bによるコメント生成
output = "./output/demo/"
input_csv = "./input/demo/ef_test_free.csv"

!python pipeline.py -c ./model_conf.yaml -i Llama-3-elyza-jp-8b $input_csv $output

[MESSAGE]:入力設定ファイルを読み込んでいます...
	model: Llama-3-elyza-jp-8b
	model_type:gguf
	model_path:./models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf
	model_url:https://huggingface.co/elyza/Llama-3-ELYZA-JP-8B-GGUF/resolve/main/Llama-3-ELYZA-JP-8B-q4_k_m.gguf
	stance:0
	prohibit_list:['17', '誤', 'タイトル', '十七', '俳句', '１７', '表題', '定型', '詳細']
	chr_num:100
	temperature:0.8
	top_P:0.8
	max_Tokens:1024
	frequency_Penalty:1.0
	presence_Penalty:1.5
	context_Length:1024
	embedding:False
	repeat_penalty:0.8
[MESSAGE] model: https://huggingface.co/elyza/Llama-3-ELYZA-JP-8B-GGUF/resolve/main/Llama-3-ELYZA-JP-8B-q4_k_m.gguf をロードしています......
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ./models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:            

In [6]:
# 自由詠でOumuamuaによるコメント生成
output = "./output/demo/"
input_csv = "./input/demo/ef_test_free.csv"

!python pipeline.py -c ./model_conf.yaml -i Oumuamua $input_csv $output

[MESSAGE]:入力設定ファイルを読み込んでいます...
	model: Oumuamua
	model_path:./models/Oumuamua-7b-instruct.Q5_K_M.gguf
	model_url:https://huggingface.co/QuantFactory/Oumuamua-7b-instruct-GGUF/resolve/main/Oumuamua-7b-instruct.Q5_K_M.gguf
	model_type:gguf
	stance:0
	prohibit_list:['17', '誤', 'タイトル', '十七', '俳句', '１７', '表題', '形式', '定型', '詳細']
	temperature:0.8
	chr_num:100
	top_P:0.8
	max_Tokens:1024
	frequency_Penalty:1.0
	presence_Penalty:1.5
	context_Length:1024
	embedding:False
	repeat_penalty:0.8
[MESSAGE]: [Oumuamua]による生成を開始します。
[MESSAGE] model: https://huggingface.co/QuantFactory/Oumuamua-7b-instruct-GGUF/resolve/main/Oumuamua-7b-instruct.Q5_K_M.gguf をロードしています......
llama_model_loader: loaded meta data with 25 key-value pairs and 291 tensors from ./models/Oumuamua-7b-instruct.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              

In [1]:
# 自由詠でUmievoによるコメント生成
output = "./output/demo/"
input_csv = "./input/demo/ef_test_free.csv"

!python pipeline.py -c ./model_conf.yaml -i Umievo $input_csv $output

[MESSAGE]:入力設定ファイルを読み込んでいます...
	model: Umievo
	model_path:umiyuki/Umievo-itr012-Gleipnir-7B
	model_type:trf
	stance:0
	prohibit_list:['17', '誤', 'タイトル', '十七', '俳句', '１７', '表題', '形式', '詳細', '季語']
	temperature:0.8
	chr_num:100
	Max_Tokens:1024
	min_new_tokens:400
	Top_P:0.95
	sleep:1
[MESSAGE]: [Umievo]による生成を開始します。
[MESSAGE] model: umiyuki/Umievo-itr012-Gleipnir-7B をロードしています......
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:06<00:00,  3.39s/it]
<<sys>>
あなたは短歌の表現や内容を評価することのできる役立つアシスタントです。
<<user>>
以下は、efさんが詠まれた短歌です。
いざよひをめづるこころはもちづきのころよりましてなほあかざりし

この短歌の表現や内容について詳細に評価した文章を出力してください。
<<assist>>
わかりました。efさんの短歌「いざよひをめづるこころはもちづきのころよりましてなほあかざりし」について評価した文章を出力します。


まず、この短歌は「夕焼けを望む心は、満月のころよりも、なお赤く輝いている」という意味になります。

この短歌は、「いざよひ」という言葉を用いることで、夕焼けという美しい自然現象を詠みました。また、「もちづき」という言葉で、満月のことを匂わせ、比較することで夕焼けが放つ強い色彩の美しさを表現しています。

さらに、「こころはもちづきのころよりましてなほあかざりし」という表現によって、夕焼けを見ることによる心の高揚感や情熱を描き出しています。「なほあかざりし」は、夕焼けの赤さがより強まっていることを強調しており、この短歌の中で最も印象的なフレーズの一つとなっています。

このように、efさんの短歌は自然の美しさと人の感情を絶妙に表現して

In [4]:
# 歌会モードで各コメントを要約
output = "./output/demo/"
input_csv = "./output/demo/ef_test_free_result.csv"

!python pipeline.py -c ./model_conf.yaml -i Gemini -m utakai  $input_csv $output

[MESSAGE]:入力設定ファイルを読み込んでいます...
	model: Gemini
	model_path:{0: 'gemini-1.5-pro', 1: 'gemini-1.5-flash', 2: 'gemini-1.0-pro', 3: 'gemini-pro'}
	model_type:gemini
	stance:0
	prohibit_list:['17', '誤', 'タイトル', '十七', '俳句', '１７', '表題']
	chr_num:100
	temperature:0.2
	sleep:60
[MESSAGE]: [Gemini]によるコメントの要約を開始します。
[MESSAGE]: プロンプトを [gemini-1.5-pro] に入力しています...
## efさんの短歌「いざよひをめづるこころはもちづきのころよりましてなほあかざりし」に対する評価のまとめ
**共通点**
* ３人の評価者全員が、この短歌から力強さ、希望、情熱といった**前向きな感情**を読み取っています。
* 特に「なほあかざりし」という表現が、作者の心の高揚や情熱を効果的に表しているという点で意見が一致しています。
**相違点**
* 「いざよひ」と「もちづき」の解釈が分かれています。
    * 1人目の評価者は、「いざよひ」を「新しいことに対する希望や期待」象徴するもの、「もちづき」を時間経過を示すものと解釈し、心境の変化に焦点を当てています。
    * 2人目の評価者は、「いざよひ」を「決意」、「もちづき」を「強い意志や執念」と解釈し、作者の強い決意や情熱に焦点を当てています。
    * 3人目の評価者は、「いざよひ」を「夕焼け」、「もちづき」を「満月」と解釈し、自然の美しさと心の高揚を対比させています。
* 全体的な解釈の方向性として、1人目は心境の変化、2人目は強い意志、3人目は自然と心の対比に重きを置いています。
**その他**
* 2人目の評価者は、「もちづき」が難解である可能性を指摘し、より分かりやすい言葉への変更を提案しています。
**まとめ**
評価者によって解釈に違いはあるものの、efさんの短歌は、力強く、希望に満ちた情熱を表現した作品であると評価されています。特に「なほあかざりし」という表現は、作者の心の状態を効果的に表現

In [5]:
# 入力結果の確認
import pandas as pd
pd.options.display.max_colwidth = 500
input_csv = "./output/demo/ef_test_free_result.csv"
df = pd.read_csv(input_csv, index_col=0)
df

,Content,Author_comment,No,Author,LLM:Llama-3-elyza-jp-8b,LLM:Oumuamua,LLM:Umievo,Utakai:Gemini
0,いざよひをめづるこころはもちづきのころよりましてなほあかざりし,NaN,1,ef,efさんの短歌「いざよひをめづるこころはもちづきのころよりましてなほあかざりし」は、以下のように評価することができます。\n\nこの短歌は、初句から三句で「いざよひをめづるこころ」という心象を描写し、「もちづきのころよりましてなほあかざりし」の四句以降で、その心象がどのように変化したかを示す構成になっています。\n\n「いざよひをめづるこころ」は、春先に花や葉が伸び始めることから、「新しいことに対する希望や期待」という意味合いに解釈できます。特に「もちづきのころよりましてなほあかざりし」は、満月を過ぎてなお赤く染まった空を描写しており、この短歌が表現している心象が、春先から更に成長や変化したことがわかります。\n\n全体として、この短歌は「新しいことに対する希望や期待」という心象が、成長や変化を経てさらに深まり広がっていく過程を描写しています。,「いざよひをめづる心は持ちづきのころよりましてなほあかざりし」という短歌は、作者のefさんが、決意や覚悟を持って何かに立ち向かう強い気持ちや、その心中に燃える情熱が伝わってくる一首です。\n\n「いざよひ」とは「いざやってみよう」という決意の言葉であり、「めづる」は「目付る」つまり見ることを意味します。心を見据えるということで、覚悟を決める際に視線が前をしっかりと見据えた状態を表しています。「もちづき」は「持ちつき」であり、持っている、あるいは固く持って離さないという意味合いがあり、ここでは強い意志や執念のようなものを表します。\n\nこの歌は、過去の自分よりも今の自分は強くなっており、何かに向かって一歩踏み出そうとしている決意を感じさせます。「まして」は以前よりも増したという意味で、心の中にある熱意が前よりも増していることを表現しています。「なほあかざりし」は輝くという意味で、その強い気持ちや情熱が光り輝いているように見える様子を表しており、力強さや希望感を感じさせてくれます。\n\n全体的に、この短歌からは、前向きでエネルギッシュなエネルギーが伝わってきます。ただ単に決意...,まず、この短歌は「夕焼けを望む心は、満月のころよりも、なお赤く輝いている」という意味になります。\n\nこの短歌は、「いざよひ」という言葉を用いることで、夕焼けという美しい自然現象を詠みました。また、「もちづき」という言葉で、満月のことを匂わせ、比較することで夕焼けが放つ強い色彩の美しさを表現しています。\n\nさらに、「こころはもちづきのころよりましてなほあかざりし」という表現によって、夕焼けを見ることによる心の高揚感や情熱を描き出しています。「なほあかざりし」は、夕焼けの赤さがより強まっていることを強調しており、この短歌の中で最も印象的なフレーズの一つとなっています。\n\nこのように、efさんの短歌は自然の美しさと人の感情を絶妙に表現しており、読む人の心を揺さぶる力があります。言葉選びや表現方法も非常に上手く、読む人に強い印象を残す短歌となっています。,## efさんの短歌「いざよひをめづるこころはもちづきのころよりましてなほあかざりし」に対する評価のまとめ\n**共通点**\n* ３人の評価者全員が、この短歌から力強さ、希望、情熱といった**前向きな感情**を読み取っています。\n* 特に「なほあかざりし」という表現が、作者の心の高揚や情熱を効果的に表しているという点で意見が一致しています。\n**相違点**\n* 「いざよひ」と「もちづき」の解釈が分かれています。\n * 1人目の評価者は、「いざよひ」を「新しいことに対する希望や期待」象徴するもの、「もちづき」を時間経過を示すものと解釈し、心境の変化に焦点を当てています。\n * 2人目の評価者は、「いざよひ」を「決意」、「もちづき」を「強い意志や執念」と解釈し、作者の強い決意や情熱に焦点を当てています。\n * 3人目の評価者は、「いざよひ」を「夕焼け」、「もちづき」を「満月」と解釈し、自然の美しさと心の高揚を対比させています。\n* 全体的な解釈の方向性として、1人目は心境の変化、2人目は強い意志、3人目は自然と心の対比に重きを置いています。\n**...
1,十六夜《いざよひ》を愛づるこころは望月のころより増してなほ飽かざりし,あなたが完璧だとわかったので、私はあなたを愛しました。それから私はあなたが完璧ではないことに気づき、あなたをさらに愛しました。,2,ef,この短歌は、十六夜（いざよひ）という月齢の月を背景に、完璧な存在と思っていた相手に対する愛情が深まる過程を描写している。\n\n「十六夜《いざよひ》」という季語は、十五夜以降の満ち欠けで、完全に新しい月が出る直前の月を指す。つまり、「望月」という完璧な存在と思っていた相手が、新たな一面を現し始める時期でもある。\n\n「十六夜《いざよひ》を愛づるこころは」で、作者の心情が表され、完全に新しい月が出る直前のこの時期に、作者の心はより強く相手に傾倒することがわかる。望月（望ましい月）という完璧な存在と思っていた相手が、新たな一面を現し始めることで、作者の愛情はますます深まり、「飽かざりし」で、作者の心は完全に満足している。\n\nこの短歌では、作者のコメント通り、「あなたが完璧だとわかったので、私はあなたを愛しました。それから私はあなたが完璧ではないことに気づき、あなたをさらに愛しました。」という過程を、十六夜（いざよひ）という月齢の月を背景に描写している。,この短歌は、「十六夜《いざよひ》」という月の呼び名と、そこから連想される情緒的な雰囲気が見事に生かされています。十六夜とは、満月から数えて16日目の月のことで、わずかに欠けた月を指します。その微妙な欠け具合が、人の心の微妙な感情や想いの深まりを表現しているかのようです。\n\n短歌の内容としては、「あなた」への愛を詠んでいますが、興味深いのは、最初は「完璧だとわかった」という視点から始まっている点です。この言葉から、相手を客観的に評価し、その上で恋心を抱いていることが伝わります。しかし、すぐに「私はあなたが完璧ではないことに気づき」と急転直下、完璧さだけではないところへの愛着や魅力を見出していく過程が描かれています。このギャップが、感情の深みと奥行きを生み出し、読者の心に残ります。\nまた、「なほ飽かざりし」という言葉が使われていることで、愛の無限大性や飽きることのない気持ちが表現されており、長く続く恋の情景が浮かんできます。月の微妙な光の加減を通して、人の心の移り変わりや深みを描いたこの短歌は、とても文学的で魅力的です。\nコメントにある「あなたが完璧だとわかったので、私は...,まず、この短歌は「十六夜《いざよひ》」という落ち着いた自然の美しさと、「望月のころ」という月の美しさが交互に描かれ、読み手の心を月明かりの中へと引き込んでいきます。短歌の中で愛を表現する上で、月の美しさは欠かせない要素の一つです。\n\n作者は、「あなたが完璧だとわかったので、私はあなたを愛しました。それから私はあなたが完璧ではないことに気づき、あなたをさらに愛しました。」とコメントしています。このコメントにより、この短歌がただの美しさの表現に留まらず、完璧な人間性を持った人を愛し、そしてその人の完璧ではない部分にも共感し、愛情を深めていく人間の心情が表現されていることがわかります。\n\n短歌の内容においても、「なほ飽かざりし」という言葉から、愛する人への愛情が尽きることがないことが表現されており、短歌の表現力が作者の心情を見事に表しています。\n\nこのように、efさんの短歌は自然の美しさと人間の心情が織り交ぜられた素晴らしい作品であり、読み手に深い感銘を与えます。コメントにより、短歌の意味がより深く理解でき、読者は作者の心情に共感し、感動を覚えることができます。,## efさんの短歌に対する評価コメントの要約と共通点・相違点\n**efさんの短歌:** \n十六夜《いざよひ》を愛づるこころは望月のころより増してなほ飽かざりし\n**efさんのコメント:** \nあなたが完璧だとわかったので、私はあなたを愛しました。それから私はあなたが完璧ではないことに気づき、あなたをさらに愛しました。\n\n### 共通点\n* 3人とも、この短歌は「十六夜」と「望月」の対比を通して、愛する対象への想いの変化と深まりを描いていると評価している。\n* 作者のコメントと短歌の内容が一致しており、「完璧さ」から「完璧ではない部分」への愛の深まりを表現している点に共感している。\n* 「なほ飽かざりし」という言葉が、尽きることのない愛情を表現している点に注目している。\n### 相違点\n* **1人目**は、短歌の内容をefさんのコメントに沿ってストレートに解釈している。\n* **2人目**は、「十六夜」のわずかに欠けた様子に、人の心の微妙な変化を重ねている。また、「完璧だとわかった」という客観的な視点から始まる点や、リズムや言葉の美しさにも触れている。...
2,友愛も敵意も遠く吹き過ぎて平均すればなだらかな線,それが人生だ、永遠の友人も永遠の敵もいない,3,ef,この短歌は、efさん独自の視点と哲学が詰まった

In [6]:
output = "./output/demo/"
input_csv = "./input/demo/ef_test_theme_sea.csv"
!python pipeline.py -c ./model_conf.yaml -i Llama-3-elyza-jp-8b --theme 海 $input_csv $output

[MESSAGE]:入力設定ファイルを読み込んでいます...
	model: Llama-3-elyza-jp-8b
	model_type:gguf
	model_path:./models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf
	model_url:https://huggingface.co/elyza/Llama-3-ELYZA-JP-8B-GGUF/resolve/main/Llama-3-ELYZA-JP-8B-q4_k_m.gguf
	stance:0
	prohibit_list:['17', '誤', 'タイトル', '十七', '俳句', '１７', '表題', '定型', '詳細']
	chr_num:100
	temperature:0.8
	top_P:0.8
	max_Tokens:1024
	frequency_Penalty:1.0
	presence_Penalty:1.5
	context_Length:1024
	embedding:False
	repeat_penalty:0.8
[MESSAGE] model: https://huggingface.co/elyza/Llama-3-ELYZA-JP-8B-GGUF/resolve/main/Llama-3-ELYZA-JP-8B-q4_k_m.gguf をロードしています......
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ./models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:            

In [7]:
output = "./output/demo/"
input_csv = "./input/demo/ef_test_theme_sea.csv"
!python pipeline.py -c ./model_conf.yaml -i Oumuamua --theme 海 $input_csv $output

[MESSAGE]:入力設定ファイルを読み込んでいます...
	model: Oumuamua
	model_path:./models/Oumuamua-7b-instruct.Q5_K_M.gguf
	model_url:https://huggingface.co/QuantFactory/Oumuamua-7b-instruct-GGUF/resolve/main/Oumuamua-7b-instruct.Q5_K_M.gguf
	model_type:gguf
	stance:0
	prohibit_list:['17', '誤', 'タイトル', '十七', '俳句', '１７', '表題', '形式', '定型', '詳細']
	temperature:0.8
	chr_num:100
	top_P:0.8
	max_Tokens:1024
	frequency_Penalty:1.0
	presence_Penalty:1.5
	context_Length:1024
	embedding:False
	repeat_penalty:0.8
[MESSAGE]: [Oumuamua]による生成を開始します。
[MESSAGE] model: https://huggingface.co/QuantFactory/Oumuamua-7b-instruct-GGUF/resolve/main/Oumuamua-7b-instruct.Q5_K_M.gguf をロードしています......
llama_model_loader: loaded meta data with 25 key-value pairs and 291 tensors from ./models/Oumuamua-7b-instruct.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              

In [8]:
output = "./output/demo/"
input_csv = "./input/demo/ef_test_theme_sea.csv"
!python pipeline.py -c ./model_conf.yaml -i Umievo --theme 海 $input_csv $output

[MESSAGE]:入力設定ファイルを読み込んでいます...
	model: Umievo
	model_path:umiyuki/Umievo-itr012-Gleipnir-7B
	model_type:trf
	stance:0
	prohibit_list:['17', '誤', 'タイトル', '十七', '俳句', '１７', '表題', '形式', '詳細', '季語']
	temperature:0.8
	chr_num:100
	Max_Tokens:1024
	min_new_tokens:400
	Top_P:0.95
	sleep:1
[MESSAGE]: [Umievo]による生成を開始します。
[MESSAGE] model: umiyuki/Umievo-itr012-Gleipnir-7B をロードしています......
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:06<00:00,  3.40s/it]
<<sys>>
あなたは短歌の表現や内容を評価することのできる役立つアシスタントです。
<<user>>
以下は「海」というお題で詠まれた短歌です。作者はefさんです。

海底の傷に海鼠（なまこ）は息づいてふうわりと降る雪を蒐める

お題が「海」であることを踏まえて、この短歌の表現や内容について詳細に評価した文章を出力してください。
<<assist>>
わかりました。お題が「海」であることを踏まえて、efさんの短歌「海底の傷に海鼠（なまこ）は息づいてふうわりと降る雪を蒐める」について評価した文章を出力します。


この短歌は、まず海底という水中の生物たちが息づく環境の中で、傷ついた海鼠がいる様子を描いています。海鼠は柔らかくて傷が付きやすい生き物であり、海底に傷を持つということは、それだけで物語性を感じさせる描写です。

さらに、そんな海鼠がふうわりと降る雪を蒐めるという情景が続きます。雪は通常、寒冷地で降るものというイメージがありますが、この短歌では海底という非日常的な場所で降る雪が、海鼠の生命力をより一層引き立てています。

また、「傷」と「雪」という対照的な要素が組み合わされることで、一層美しく表現されています。傷があることで生命力を強く感じ、雪が降

In [ ]:
# 歌会モードで各コメントを要約
output = "./output/demo/"
input_csv = "./output/demo/ef_test_theme_sea_result.csv"

!python pipeline.py -c ./model_conf.yaml -t 海 -i Gemini -m utakai  $input_csv $output

In [ ]:
# 入力結果の確認
import pandas as pd
pd.options.display.max_colwidth = 500
input_csv = "./output/demo/ef_test_theme_sea_result.csv"
df = pd.read_csv(input_csv, index_col=0)
df

In [ ]:
# 題詠「海」・human_assistあり
output = "./output/demo/"
input_csv = "./input/demo/ef_test_theme_sea_human_comment.csv"

!python pipeline.py -c ./model_conf.yaml -i Llama-3-elyza-jp-8b --theme 海 $input_csv $output

In [ ]:
# 題詠「海」・human_assistあり
output = "./output/demo/"
input_csv = "./input/demo/ef_test_theme_sea_human_comment.csv"

!python pipeline.py -c ./model_conf.yaml -i Oumuamua --theme 海 $input_csv $output

In [ ]:
# 題詠「海」・human_assistあり
output = "./output/demo/"
input_csv = "./input/demo/ef_test_theme_sea_human_comment.csv"

!python pipeline.py -c ./model_conf.yaml -i Umievo --theme 海 $input_csv $output

In [ ]:
# 入力結果の確認
import pandas as pd
pd.options.display.max_colwidth = 500
input_csv = "./output/demo/ef_test_theme_sea_human_comment_result.csv"
df = pd.read_csv(input_csv, index_col=0)
df

In [ ]:
output = "./output/demo/"
input_csv = "./output/demo/ef_test_free_result.csv"

!python pipeline.py -c ./model_conf.yaml -i Gemini -m utakai  $input_csv $output